import urllib.request

url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
week_nums = [210109, 210116, 210123, 210130, 210206, 210213, 210220, 210227, 210306, 210313, 210320,
210327, 210403, 210410, 210417]

for week_num in week_nums:
    urllib.request.urlretrieve(f"http://web.mta.info/developers/data/nyct/turnstile/turnstile_{week_num}.txt",
                               f"data/turnstile_{week_num}.txt")

/Users/philliprichardson/Metis/Module 1/mta.db

In [6]:
from sqlalchemy import create_engine
import pandas as pd


engine = create_engine("sqlite:///Users/philliprichardson/Metis/Module 1/mta.db")
all_data = pd.read_sql('with mta as(select *, row_number() over (PARTITION by CA, UNIT,  STATION,  SCP, date, time order by CA, UNIT,  STATION,  SCP, date, time) as dupes from mta_data) select *, case when (time between 11 and 12) or (time between 19 and 20) then 1 else 0 end as rush_hour, LAG ( entries, 1, 0 ) OVER ( partition by CA, UNIT,  STATION,  SCP ORDER BY date, time)  as entered, LAG ( exits, 1, 0 ) OVER ( partition by CA, UNIT,  STATION,  SCP ORDER BY date, time) as exited from mta where dupes = 1;', engine)
all_data2 = pd.read_sql('SELECT * FROM NYC_Covid;', engine)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [32]:
import datetime

all_data.head()
# type(all_data['DATE'][0])

all_data['DOW'] = pd.to_datetime(all_data['DATE'])
# type(all_data['DOW'][0])
all_data['DOW1'] = all_data['DOW'].dt.strftime('%a')
all_data.head(100)

# all_data['DOW'] = pd.to_datetime(all_data['DATE'], format = '%M/%d/%y')

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,dupes,rush_hour,entered,exited,DOW,DOW1
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/02/2021,03:00:00,REGULAR,7511653,2558871,1,0,0,0,2021-01-02,Sat
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/02/2021,07:00:00,REGULAR,7511655,2558877,1,0,7511653,2558871,2021-01-02,Sat
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/02/2021,11:00:00,REGULAR,7511677,2558930,1,1,7511655,2558877,2021-01-02,Sat
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/02/2021,15:00:00,REGULAR,7511766,2558968,1,0,7511677,2558930,2021-01-02,Sat
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/02/2021,19:00:00,REGULAR,7511912,2558999,1,1,7511766,2558968,2021-01-02,Sat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/17/2021,23:00:00,REGULAR,7518733,2561962,1,0,7518679,2561953,2021-01-17,Sun
96,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/18/2021,03:00:00,REGULAR,7518736,2561965,1,0,7518733,2561962,2021-01-18,Mon
97,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/18/2021,07:00:00,REGULAR,7518738,2561988,1,0,7518736,2561965,2021-01-18,Mon
98,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/18/2021,11:00:00,REGULAR,7518784,2562082,1,1,7518738,2561988,2021-01-18,Mon


What stations experience the most traffic during standard commute hours? (7-11) (15-19)

Question: We want to better allocate our employees during rush hour/standard commute hours, what stations represent the highest priority during the rush hour time frame? 

How are we measuring traffic? Is there a difference between exits and entrances? Does the # of lines impact traffic?

Do Covid Cases in NYC have a relationship (given 2 week lag) with ridership?

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3137719 entries, 0 to 3137718
Data columns (total 11 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   CA        object
 1   UNIT      object
 2   SCP       object
 3   STATION   object
 4   LINENAME  object
 5   DIVISION  object
 6   DATE      object
 7   TIME      object
 8   DESC      object
 9   ENTRIES   int64 
 10  EXITS     int64 
dtypes: int64(2), object(9)
memory usage: 263.3+ MB
